# Grupo 03 - Desafío 01
## Estrategia de construcción
>**OBJETIVO:** Crear un df de post-producción que luego se transformará en producción para realizar análisis del dataset.
> * Se trabajará depurando cada columna del dataset con la finalidad de reconstruir datos perdidos y/o crear nuevas columnas que serán utilizadas a posteriori en la fase de análisis.

### Instalación de librerías

In [ ]:
#!pip install unidecode

### Importación de librerías

In [ ]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

### Carga e inicialización

In [ ]:
# Carga de dataset inicial

df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
# Definición de cantidad de filas a cargar en caso de realizar una prueba acotada

ca_filas = 40
#ca_filas = 121220

In [ ]:
# Reemplado de NaN's para comenzar a estandarizar los datos ingresados en data frame de inicio

df = g3.reemplaza_nan('sin datos',df)

In [ ]:
# Verificación del formato del data frame incicial

df.shape

### Armado inicial de data frames operativos
Este bloque se reserva para realizar la precarga inicial de distintos data frames que se utilizarán a lo largo del notebook.
* pospo -- df de posproducción
* df_recup -- df para persistir los elementos recuperados


In [ ]:
# Dadtaframe de posproducción. Tras depurar todas las regex, este DF será la base de análisis de negocio
# Se ejecuta una única vez y se le van incorporando las columnas depuradas.
# Tener en cuenta que el DF pospo definitivo tiene que tener la misma cantidad de filas que el DF base len(df)

pospo = g3.generar_df_posproduccion(ca_filas)
pospo.shape
#pospo

In [ ]:
# Definición de df que permite registrar los valores recuperados de cada columna

df_recup = pd.DataFrame()
df_recup.shape

### Procesamiento de carga de columnas depuradas en data frame post-produccion 'pospo'
> *** Modo de uso:** 
>* En este markdown se vuelcan las funciones que generan y agregan columnas depuradas dentro de pospo
>* La codificación de desarrollo se realiza en notebook de **Desarrollo** y una vez terminado se empaqueta dentro de una función consolidadora para automatizar tareas de carga.



In [ ]:
# Depuración 'ambientes' en 'description', persistencia de columna en pospro y recup

def recuperar_ambientes(pospo, recup):
    pattern = r'(\d*\w*)\s*amb'  
    
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    
    df_temp = g3.limpiar_columna_x_clave(dic, df1)
    recuperados = df_temp[pd.notnull(df_temp[0])].count() # selecciona unicamente los recuperados del regex
    recup = g3.registrar_recupero(int(recuperados),'r_ambientes', recup)
    pospo = g3.agregar_columna('r_ambiente', pospo, df_temp)
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_ambientes(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
# Depuración 'precio usd' en 'description', persistencia de columna en pospro y recup

def recuperar_usd(pospo, recup):
    pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado(df1, 'r_usd') # Genera DF acotado con los valores recuperados unicamente

    recup = g3.registrar_recupero(len(df2),'r_usd', recup)

    pospo = pd.merge(pospo, df2, on='indice', how='left')
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_usd(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
# Depuración 'metros2' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_metros(pospo, recup):
    pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_metros2') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_metros2')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_metros2', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_metros(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
# Depuración 'frente' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_frente(pospo, recup):
    pattern = r"(frente)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_frente')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_frente')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_frente', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_frente(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
# Depuración 'contrafrente' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_contrafrente(pospo, recup):
    pattern = r"(contrafrente)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    df2 = g3.obtener_df_indexado_simple(df1, 'r_contrafrente')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_contrafrente')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_contrafrente', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_contrafrente(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
# Depuración 'expensas' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_expensas(pospo, recup):
    pattern = r"(expensas)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_expensas')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_expensas')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
        
    recup = g3.registrar_recupero(len(df5),'r_expensas', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup



In [ ]:
pospo, df_recup = recuperar_expensas(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
def recuperar_garage(pospo, recup):
    pattern = r"(garage)|(cochera)" # garage
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_garage') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_garage')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_garage', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_garage(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
def recuperar_amenities(pospo, recup):
    pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)" # garage
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_amenities') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_amenities')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_amenities', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [ ]:
pospo, df_recup = recuperar_amenities(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

In [ ]:
df_recup.T

### Incorporación de características originales
* Dichas características se extraen del data frame 'df' y se persisten en 'pospo'.
* Las características seleccionadas fueron analizadas previamente y solo se persistirán las necesarias.